# Content Understanding via Landing Zone Gateway

**Access Content Understanding as a governed, centralized service through your Landing Zone APIM.**

## The Big Picture

In the [main deploy.ipynb](../deploy.ipynb), you deployed Content Understanding directly and called it with AAD tokens. That's great for learning!

For **production teams**, you want:

```
Team A ─┐
Team B ─┼─→ Landing Zone APIM ─→ Shared CU Instance
Team C ─┘        ↓
           Rate limits, Quotas, Logging
```

## Why Govern CU Access?

| Challenge | Without Gateway | With Gateway |
|-----------|----------------|---------------|
| **Cost Control** | Each team deploys own CU | Shared instance, quota per team |
| **Rate Limiting** | CU default limits only | Custom limits per subscription |
| **Observability** | Scattered logs | Unified logging in APIM |
| **Security** | Each team manages auth | Central auth via API keys |

## Prerequisites

- ✅ **Lab 1A** Landing Zone deployed (APIM)
- ✅ **Lab 9** Content Understanding deployed ([main deploy.ipynb](../deploy.ipynb))
- ✅ `.env` with `APIM_URL`, `APIM_KEY`

In [ ]:
import os, subprocess, json, re, time, requests
from pathlib import Path
from IPython.display import display, Markdown

# Load environment
for env_path in [Path("../../.env"), Path("../.env"), Path(".env")]:
    if env_path.exists():
        for line in env_path.read_text().splitlines():
            if '=' in line and not line.startswith('#'):
                k, v = line.split('=', 1)
                os.environ[k.strip()] = v.strip()

# Landing Zone config
APIM_URL = os.environ.get("APIM_URL", "")
APIM_KEY = os.environ.get("APIM_KEY", "")
APIM_NAME = re.match(r'https://([^.]+)\.', APIM_URL).group(1) if APIM_URL else ""
GATEWAY_URL = APIM_URL.rsplit('/', 1)[0] if APIM_URL else ""  # Remove /openai suffix

# Get resource group
result = subprocess.run(
    ["az", "apim", "list", "--query", f"[?name=='{APIM_NAME}'].resourceGroup", "-o", "tsv"],
    capture_output=True, text=True
)
APIM_RG = result.stdout.strip()

print("📋 Landing Zone Configuration")
print("==============================")
print(f"✅ APIM Name: {APIM_NAME}")
print(f"✅ Gateway URL: {GATEWAY_URL}")
print(f"✅ Resource Group: {APIM_RG}")

In [ ]:
# Get Content Understanding deployment from Lab 9
CU_RG = "content-understanding-lab-rg"

result = subprocess.run(
    f'az deployment group show -g "{CU_RG}" -n spoke --query properties.outputs -o json',
    shell=True, capture_output=True, text=True
)

if result.returncode != 0:
    print("❌ CU deployment not found. Run the main deploy.ipynb first!")
    print(f"   Error: {result.stderr}")
else:
    outputs = json.loads(result.stdout)
    CU_ENDPOINT = outputs['contentUnderstandingEndpoint']['value']
    CU_ACCOUNT_NAME = outputs['accountName']['value']
    
    # Construct resource ID
    sub_id = subprocess.run('az account show --query id -o tsv', shell=True, capture_output=True, text=True).stdout.strip()
    CU_RESOURCE_ID = f"/subscriptions/{sub_id}/resourceGroups/{CU_RG}/providers/Microsoft.CognitiveServices/accounts/{CU_ACCOUNT_NAME}"
    
    print("📋 Content Understanding Configuration")
    print("=======================================")
    print(f"✅ CU Endpoint: {CU_ENDPOINT}")
    print(f"✅ CU Account: {CU_ACCOUNT_NAME}")

---

## Step 1: Add CU API to Landing Zone APIM

Deploy the Bicep template that adds `/cu/*` endpoints to your existing APIM gateway.

**What gets created:**
- `/cu/analyzers/{analyzer}:analyze` - Submit content for analysis
- `/cu/analyzers/{analyzer}/results/{id}` - Get analysis results
- `/cu/analyzers` - List available analyzers
- `/cu/defaults` - Get/update model defaults

**Governance policies:**
- 30 calls/minute per subscription
- 1000 calls/day quota per subscription
- Managed identity auth to CU backend
- Correlation IDs for tracing

In [ ]:
# Deploy CU API to APIM
print("🚀 Adding Content Understanding API to Landing Zone APIM...")
print(f"   APIM: {APIM_NAME}")
print(f"   CU Backend: {CU_ENDPOINT}")
print("")

deploy_cmd = f'''az deployment group create \
    -g "{APIM_RG}" \
    --template-file deployment.bicep \
    -p apimName="{APIM_NAME}" \
    -p cuEndpoint="{CU_ENDPOINT}" \
    -p cuResourceId="{CU_RESOURCE_ID}" \
    -p cuResourceGroup="{CU_RG}" \
    -o table'''

!{deploy_cmd}

In [ ]:
# Get deployment outputs
result = subprocess.run(
    f'az deployment group show -g "{APIM_RG}" -n deployment --query properties.outputs -o json',
    shell=True, capture_output=True, text=True
)

if result.returncode == 0:
    outputs = json.loads(result.stdout)
    CU_GATEWAY_URL = outputs['gatewayUrl']['value']
    
    print("✅ Content Understanding API added to Landing Zone!")
    print("")
    print(f"🌐 Gateway URL: {CU_GATEWAY_URL}")
    print(f"🔑 API Key: {APIM_KEY[:8]}...{APIM_KEY[-4:]}" if len(APIM_KEY) > 12 else f"🔑 API Key: {APIM_KEY}")
    print("")
    print("📡 Available endpoints:")
    print(f"   POST {CU_GATEWAY_URL}/analyzers/{{analyzer}}:analyze")
    print(f"   GET  {CU_GATEWAY_URL}/analyzers/{{analyzer}}/results/{{id}}")
    print(f"   GET  {CU_GATEWAY_URL}/analyzers")
else:
    CU_GATEWAY_URL = f"{GATEWAY_URL}/cu"
    print(f"⚠️ Could not get outputs, using: {CU_GATEWAY_URL}")

---

## Step 2: Test Governed CU Access

Now teams can access CU using just an **API key** - no AAD tokens needed!

In [17]:
# Wait for RBAC propagation
print("⏳ Waiting for RBAC propagation (30s)...")
time.sleep(30)
print("✅ Ready!")

⏳ Waiting for RBAC propagation (30s)...
✅ Ready!


In [ ]:
class GoverningCUClient:
    """Content Understanding client that goes through the Landing Zone gateway."""
    
    def __init__(self, gateway_url, api_key, api_version="2025-11-01"):
        self.gateway_url = gateway_url.rstrip('/')
        self.api_key = api_key
        self.api_version = api_version
    
    def _headers(self):
        return {
            "api-key": self.api_key,
            "Content-Type": "application/json"
        }
    
    def analyze(self, analyzer, inputs, poll_interval=2, max_wait=600):
        """Submit content for analysis and poll for results."""
        url = f"{self.gateway_url}/analyzers/{analyzer}:analyze?api-version={self.api_version}"
        
        # Submit analysis request
        resp = requests.post(url, headers=self._headers(), json={"inputs": inputs})
        
        if not resp.ok:
            return {"error": f"{resp.status_code}: {resp.text}"}
        
        # Get operation location for polling
        op_url = resp.headers.get('Operation-Location')
        if not op_url:
            return {"error": "No Operation-Location header"}
        
        # Replace direct CU URL with gateway URL for polling
        # The backend returns its own URL, we need to route through APIM
        if 'cognitiveservices.azure.com' in op_url:
            # Extract the path after /contentunderstanding
            path = op_url.split('/contentunderstanding')[1]
            op_url = f"{self.gateway_url}{path}"
        
        # Poll for result
        start = time.time()
        while time.time() - start < max_wait:
            r = requests.get(op_url, headers=self._headers())
            if not r.ok:
                return {"error": f"Poll failed: {r.status_code}"}
            
            result = r.json()
            status = result.get('status', '')
            
            if status == 'Succeeded':
                return result
            if status in ['Failed', 'Cancelled']:
                return {"error": result}
            
            time.sleep(poll_interval)
        
        return {"error": "Timeout"}
    
    def list_analyzers(self):
        """List available analyzers."""
        url = f"{self.gateway_url}/analyzers?api-version={self.api_version}"
        resp = requests.get(url, headers=self._headers())
        return resp.json() if resp.ok else {"error": resp.text}


# Create governed CU client
cu = GoverningCUClient(CU_GATEWAY_URL, APIM_KEY)
print("✅ Governed CU client ready!")
print(f"   Gateway: {CU_GATEWAY_URL}")
print(f"   Auth: API Key (no AAD tokens needed!)")

In [19]:
# Test: List available analyzers
print("📋 Available Analyzers (via Gateway):")
analyzers = cu.list_analyzers()

if 'error' in analyzers:
    print(f"❌ Error: {analyzers['error']}")
else:
    for a in analyzers.get('value', []):
        print(f"   • {a.get('analyzerId', 'unknown')}: {a.get('description', '')[:50]}...")

📋 Available Analyzers (via Gateway):
   • prebuilt-audio: Transcribe conversations....
   • prebuilt-audioSearch: Transcribe conversations and extract summaries....
   • prebuilt-bankStatement.us: Extract bank statement US document fields....
   • prebuilt-callCenter: Analyze call center conversations to extract trans...
   • prebuilt-check.us: Extract check US document fields....
   • prebuilt-contract: Extract contract document fields....
   • prebuilt-creditCard: Extract credit card document fields....
   • prebuilt-creditMemo: Extract credit memo document fields....
   • prebuilt-document: Extract various content and layout elements such a...
   • prebuilt-documentFields: Propose key-value document fields....
   • prebuilt-documentFieldSchema: Propose field schema for document content....
   • prebuilt-documentSearch: Extract various content and layout elements such a...
   • prebuilt-healthInsuranceCard.us: Extract health insurance card US document fields....
   • prebuilt-idDocum

---

## Step 3: Analyze Document Through Gateway

Same NASA document as the main lab, but now accessed through governed APIM!

In [ ]:
# Sample NASA document
SAMPLE_PDF_URL = "https://ntrs.nasa.gov/api/citations/19720018364/downloads/19720018364.pdf"
SAMPLE_TITLE = "Apollo 14 Mission Report"

print(f"📄 Analyzing: {SAMPLE_TITLE}")
print(f"   URL: {SAMPLE_PDF_URL[:50]}...")
print(f"   Via: Landing Zone Gateway")
print("")

doc_result = cu.analyze("prebuilt-layout", [{"url": SAMPLE_PDF_URL}])

if 'error' in doc_result:
    print(f"❌ Error: {doc_result['error']}")
else:
    contents = doc_result.get('result', {}).get('contents', [])
    markdown = contents[0].get('markdown', '') if contents else ''
    print(f"✅ Extracted {len(markdown):,} characters")
    print(f"   Content blocks: {len(contents)}")

In [24]:
# Display extracted content
if 'error' not in doc_result and markdown:
    preview = markdown[:1500] + ("\n\n*... (truncated)*" if len(markdown) > 1500 else "")
    display(Markdown(f"### 📝 Extracted Content (via Governed Gateway)\n\n{preview}"))

### 📝 Extracted Content (via Governed Gateway)

2 (mix)

NASA CR-120916

(NASA-CR-120916) - DESIGN OF A TF34 TURBOFAN
MIXER FOR REDUCTION OF FLAP IMPINGEMENT
NOISE Final Report A. Chamay, et al
(General Electric Co.) 2 Feb. 1972 131 p

N72-26014

Unclas
CSCL 21E G3/02 32002


![NASA](figures/1.1)


DESIGN OF A TF34 TURBOFAN MIXER FOR
REDUCTION OF FLAP IMPINGEMENT NOISE

FINAL REPORT

by A. Chamay, D.P. Edkins, R.B. Mishler and W.S. Clapper
Reproduced by
NATIONAL TECHNICAL
INFORMATION SERVICE
U S Department of Commerce
Springfield VA 22151

GENERAL ELECTRIC COMPANY
AIRCRAFT ENGINE GROUP
LYNN, MASSACHUSETTS/CINCINNATI OHIO

Prepared for
NATIONAL AERONAUTICS AND SPACE ADMINISTRATION
February 2, 1972

NASA Lewis Research Center
Cleveland, Ohio
N.E. Samanich
Project Manager

CONTRACT NAS 3-14338 Modification 2

RECEIVED
JUN 1972
GISA STI FACILITY
INZUT BRAMEN
67 8 9 101112 13 14 1J

/3/2

<!-- PageBreak -->

<!-- PageHeader: NASA CR-120916 -->


# FINAL REPORT

DESIGN OF A TF34 TURBOFAN MIXER FOR REDUCTION
OF FLAP IMPINGEMENT NOISE

by

A. Chamay, D. P. Edkins, R. B. Mishler and W. S. Clapper

General Electric Company
Aircraft Engine Group
Lynn, Massachusetts/Cincinnati, Ohio

prepared for
NATIONAL AERONAUTICS AND SPACE ADMINISTRATION
February 2, 1972

CONTRACT NAS3-14338 Modification 2

NASA Lewis Research Center
Cleveland, Ohio
N. E. Samanich - Project Manager

<!-- PageBreak -->

<!-- PageHeader: PRECEDING PAGE BLANK NOT FILMED -->


## TABLE OF CONTENTS


<table>
<tr>
<th></th>
<th></th>
<th>Page</th>
</tr>
<tr>
<td>ABSTRAC

*... (truncated)*

---

## Summary

You've extended your **Landing Zone AI Gateway** to provide **governed Content Understanding access**!


In [ ]:
# Cleanup (uncomment to remove CU API from APIM)
# !az apim api delete -g "{APIM_RG}" -n "{APIM_NAME}" --api-id content-understanding-api --yes
# print("🗑️ CU API removed from Landing Zone")